In [28]:
import json
import os
import pandas as pd
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, balanced_accuracy_score, f1_score
from sklearn.neighbors import KNeighborsClassifier
from sklearn.preprocessing import StandardScaler

In [29]:
csv_root_path = "../datasets/intermediate-features/"

In [30]:
def calculate_scores(
    targets,
    predictions,
    model_name=None,
    feature_identifier=None,
    number_of_samples=None,
):
    accuracy = accuracy_score(targets, predictions)
    balanced_accuracy = balanced_accuracy_score(targets, predictions)
    f1_weighted = f1_score(targets, predictions, average="weighted")
    f1_micro = f1_score(targets, predictions, average="micro")
    f1_macro = f1_score(targets, predictions, average="macro")
    if model_name is None or feature_identifier is None:
        print(f"Accuracy: {accuracy}")
        print(f"Balanced accuracy: {balanced_accuracy}")
        print(f"F1-weighted: {f1_weighted}")
        print(f"F1-micro: {f1_micro}")
        print(f"F1-macro: {f1_macro}")
    else:
        print(f"model_name={model_name}, feature_identifier={feature_identifier}")
        with open("../runs/metrics.txt", "a") as detaillog:
            line = {}
            line["model_name"] = model_name
            line["feature_identifier"] = feature_identifier
            line["number_of_samples"] = number_of_samples
            line["accuracy"] = accuracy
            line["balanced_accuracy"] = balanced_accuracy
            line["f1_weighted"] = f1_weighted
            line["f1_micro"] = f1_micro
            line["f1_macro"] = f1_macro
            json.dump(line, detaillog, indent=2)


def calculate_scores_for_file(csv_path, number_of_samples=None):
    df_full = pd.read_csv(csv_path, index_col=0)
    df_train = df_full[df_full["set"] == "train"]
    df_valid = df_full[df_full["set"] == "valid"]
    df_test = df_full[df_full["set"] == "test"]

    if number_of_samples is not None:
        df_train = df_train.groupby("target").head(number_of_samples)

    df_train_features = df_train.loc[:, ~df_train.columns.isin(["target", "set"])]
    df_valid_features = df_valid.loc[:, ~df_valid.columns.isin(["target", "set"])]
    df_test_features = df_test.loc[:, ~df_test.columns.isin(["target", "set"])]

    scaler = StandardScaler()
    train_features = scaler.fit_transform(df_train_features)
    valid_features = scaler.transform(df_valid_features)
    test_features = scaler.transform(df_test_features)

    train_targets = df_train["target"].to_numpy()
    valid_targets = df_valid["target"].to_numpy()
    test_targets = df_test["target"].to_numpy()

    model_lr = LogisticRegression(max_iter=10000)
    model_lr.fit(train_features, train_targets)

    valid_pred = model_lr.predict(valid_features)
    test_pred = model_lr.predict(test_features)
    calculate_scores(test_targets, test_pred, "lr", csv_path, number_of_samples)

    scores = {}
    max_neighbors = 10
    if number_of_samples is not None:
        max_neighbors = min(max_neighbors, 1 + number_of_samples)

    for k in range(1, max_neighbors):
        model_knn = KNeighborsClassifier(n_neighbors=k)
        model_knn.fit(train_features, train_targets)
        valid_pred = model_knn.predict(valid_features)
        score = f1_score(valid_targets, valid_pred, average="weighted")
        scores[k] = score
    # print(f"Scores {scores}")

    best_k = max(scores, key=scores.get)
    model_knn = KNeighborsClassifier(n_neighbors=best_k)
    model_knn.fit(train_features, train_targets)
    test_pred = model_knn.predict(test_features)
    calculate_scores(test_targets, test_pred, "knn", csv_path, number_of_samples)

In [31]:
csv_paths = os.listdir(path=csv_root_path)
df_comparison = pd.DataFrame()

for i, path in enumerate(csv_paths):
    df_full = pd.read_csv(os.path.join(csv_root_path, path), index_col=0)
    df_comparison[f"target_{i}"] = df_full["target"]
    df_comparison[f"set_{i}"] = df_full["set"]

In [32]:
df_comparison[df_comparison.columns[0::2]].T.duplicated()  # verify if split is the same
# first row must be False, all other True

target_0     False
target_1      True
target_2      True
target_3      True
target_4      True
target_5      True
target_6      True
target_7      True
target_8      True
target_9      True
target_10     True
target_11     True
target_12     True
target_13     True
target_14     True
target_15     True
dtype: bool

In [33]:
df_comparison[df_comparison.columns[1::2]].T.duplicated()  # verify if split is the same
# first row must be False, all other True

set_0     False
set_1      True
set_2      True
set_3      True
set_4      True
set_5      True
set_6      True
set_7      True
set_8      True
set_9      True
set_10     True
set_11     True
set_12     True
set_13     True
set_14     True
set_15     True
dtype: bool

In [34]:
for i, path in enumerate(csv_paths):
    csv_path = os.path.join(csv_root_path, path)
    for number_of_samples in [1, 3, 10, 30, None]:
        calculate_scores_for_file(csv_path, number_of_samples)

model_name=lr, feature_identifier=../datasets/intermediate-features/Cassava-ResNet50-ImageNet_SSL.csv
model_name=knn, feature_identifier=../datasets/intermediate-features/Cassava-ResNet50-ImageNet_SSL.csv
model_name=lr, feature_identifier=../datasets/intermediate-features/Cassava-ResNet50-ImageNet_SSL.csv
model_name=knn, feature_identifier=../datasets/intermediate-features/Cassava-ResNet50-ImageNet_SSL.csv
model_name=lr, feature_identifier=../datasets/intermediate-features/Cassava-ResNet50-ImageNet_SSL.csv
model_name=knn, feature_identifier=../datasets/intermediate-features/Cassava-ResNet50-ImageNet_SSL.csv
model_name=lr, feature_identifier=../datasets/intermediate-features/Cassava-ResNet50-ImageNet_SSL.csv
model_name=knn, feature_identifier=../datasets/intermediate-features/Cassava-ResNet50-ImageNet_SSL.csv
model_name=lr, feature_identifier=../datasets/intermediate-features/Cassava-ResNet50-ImageNet_SSL.csv
model_name=knn, feature_identifier=../datasets/intermediate-features/Cassava-R